In [1]:
import sympy as sp

## 坐标及参数定义
相机内参矩阵：
$$\mathbf{K} = \begin{bmatrix} \alpha_x & 0 & x_0 \\ 0 & \alpha_y & y_0 \\ 0 & 0 & 1\end{bmatrix}$$
定义归一化平面坐标$\mathbf{p}_n$、考虑畸变的归一化坐标$\mathbf{p}^d_n$和场景坐标（即相机坐标系下的坐标）$\mathbf{p}_s$，且有：
$$\mathbf{p}_n = \begin{bmatrix} x_n\\ y_n\\ 1\end{bmatrix}= \frac{1}{z_s}\mathbf{p}_s= \frac{1}{z_s}\begin{bmatrix}x_s \\y_s \\z_s\end{bmatrix}$$
$$\mathbf{p}^d_n = \begin{bmatrix}
x_n(1 + k_1 r^2_n + k_2 r^4_n) + 2 p_1 x_n y_n + p_2 (r^2_n + 2 x^2_n) \\
y_n(1 + k_1 r^2_n + k_2 r^4_n) + p_1(r^2_n + 2 y^2_n) + 2p_2 x_n y_n \\
1
\end{bmatrix}$$
对畸变过程，记为：
$$\mathbf{p}^d_n = D(\mathbf{p}_n)$$
定义世界坐标$\mathbf{p}_w$，有：
$$\mathbf{p}_s =
\mathbf{T}_{sw} \mathbf{p}_w$$
且：
$$\begin{align}\mathbf{T}_{cw} & = \mathrm{Exp}(\xi) \\\xi &= \begin{bmatrix}\rho   \\\phi\end{bmatrix}\end{align}$$
$\rho$代表平移、$\phi$代表旋转。

定义像素坐标$\mathbf{p}_p$和考虑畸变的像素坐标$\mathbf{p}^d_p$，且有：
$$\mathbf{p}^d_p =
\mathbf{K} \mathbf{p}^d_n$$
为了方便，若$\mathbf{p}$是齐次坐标，则求导的时候记$\mathbf{p}$为非齐次坐标，例如：
$$\mathbf{p}^d_n = \begin{bmatrix}x_n^p \\y_n^p\end{bmatrix},\mathbf{p}^d_p = \begin{bmatrix}x_p^p \\y_p^p\end{bmatrix}$$
## 优化问题
我们想通过非线性优化调整每幅图像与相机的相对位姿$\xi$、3D点的世界坐标$\mathbf{P}_{w}$。
记$\mathbf{m} = \begin{bmatrix}
u_{obs} \\
v_{obs}
\end{bmatrix}$为实际测量到的像素坐标，$\mathbf{r}$为残差，有：
$$\mathbf{r} =\mathbf{p}^d_p - \mathbf{m} =  \begin{bmatrix} x^d_p - u_{obs}\\y^d_p - v_{obs}\end{bmatrix}$$
为了优化这些参数，我们需要推导残差对参数的雅克比矩阵$\frac{\partial \mathbf{r}}{\partial \mathbf{\xi}} $、$\frac{\partial \mathbf{r}}{\partial \mathbf{P}_{w}}$。

In [19]:
alpha_x, alpha_y, x0, y0 = sp.symbols('alpha_x, alpha_y, x_0, y_0')
k1, k2, p1, p2 = sp.symbols('k1, k2, p1, p2')
xn, yn, rn, pn = sp.symbols('x_n, y_n, r_n, p_n')
xdn, ydn, pdn = sp.symbols('x_n^d, y_n^d, p_n^d')
K = sp.Matrix([[alpha_x, 0, x0], [0, alpha_y, y0], [0, 0, 1]])
pn = sp.Matrix([[xn], [yn], [1]])
pdn = sp.Matrix([[xdn], [ydn], [1]])
display(K, pn, pdn)

Matrix([
[alpha_x,       0, x_0],
[      0, alpha_y, y_0],
[      0,       0,   1]])

Matrix([
[x_n],
[y_n],
[  1]])

Matrix([
[x_n^d],
[y_n^d],
[    1]])

In [20]:
xdn = xn * (1+k1*rn**2 + k2*rn**4) + 2*p1*xn*yn + p2*(rn**2 + 2*xn**2)
ydn = yn * (1+k1*rn**2 + k2*rn**4) + p1*(rn**2 + 2*yn**2) + 2*p2*xn*yn
pdn = sp.Matrix([[xdn], [ydn], [1]])
pdp = K * pdn

pdp.row_del(2)
param_vector = sp.Matrix([[alpha_x], [alpha_y], [x0], [y0], [k1], [k2], [p1], [p2]])

Matrix([
[2*p1*x_n*y_n + p2*(r_n**2 + 2*x_n**2) + x_n*(k1*r_n**2 + k2*r_n**4 + 1),                                                                       0, 1, 0, alpha_x*r_n**2*x_n, alpha_x*r_n**4*x_n,           2*alpha_x*x_n*y_n, alpha_x*(r_n**2 + 2*x_n**2)],
[                                                                      0, p1*(r_n**2 + 2*y_n**2) + 2*p2*x_n*y_n + y_n*(k1*r_n**2 + k2*r_n**4 + 1), 0, 1, alpha_y*r_n**2*y_n, alpha_y*r_n**4*y_n, alpha_y*(r_n**2 + 2*y_n**2),           2*alpha_y*x_n*y_n]])

下面推导$\frac{\partial \mathbf{r}}{\partial \mathbf{\xi}}$
由于：
$$\frac{\partial \mathbf{r}}{\partial \mathbf{\xi}} =\frac{\partial \mathbf{p}^d_p}{\partial \mathbf{p}_{n}} \frac{\partial \mathbf{p}_n}{\partial \mathbf{p}_{s}} \frac{\partial \mathbf{p}_s}{\partial \mathbf{\xi}} $$

故只需分别推导$\frac{\partial \mathbf{p}^d_p}{\partial \mathbf{p}_{n}} $、
$\frac{\partial \mathbf{p}_n}{\partial \mathbf{p}_{s}} $、
$\frac{\partial \mathbf{p}_s}{\partial \mathbf{\xi}} $

In [21]:
pn.row_del(2)

In [24]:
pdp.jacobian(pn)

Matrix([
[alpha_x*(k1*r_n**2 + k2*r_n**4 + 2*p1*y_n + 4*p2*x_n + 1),                                          2*alpha_x*p1*x_n],
[                                         2*alpha_y*p2*y_n, alpha_y*(k1*r_n**2 + k2*r_n**4 + 4*p1*y_n + 2*p2*x_n + 1)]])

故$$\frac{\partial \mathbf{p}^d_p}{\partial \mathbf{p}_{n}} = \left[\begin{matrix}\alpha_{x} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 2 p_{1} y_{n} + 4 p_{2} x_{n} + 1\right) & 2 \alpha_{x} p_{1} x_{n}\\2 \alpha_{y} p_{2} y_{n} & \alpha_{y} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 4 p_{1} y_{n} + 2 p_{2} x_{n} + 1\right)\end{matrix}\right]$$

In [28]:
xs, ys, zs, ps = sp.symbols('x_s, y_s, z_s, p_s')
ps = sp.Matrix([[xs], [ys], [zs]])
xn = xs / zs
yn = ys / zs
pn = sp.Matrix([[xn], [yn]])
pn.jacobian(ps)

Matrix([
[1/z_s,     0, -x_s/z_s**2],
[    0, 1/z_s, -y_s/z_s**2]])

故
$$\frac{\partial \mathbf{p}_n}{\partial \mathbf{p}_{s}} = \left[\begin{matrix}\frac{1}{z_{s}} & 0 & - \frac{x_{s}}{z_{s}^{2}}\\0 & \frac{1}{z_{s}} & - \frac{y_{s}}{z_{s}^{2}}\end{matrix}\right]$$
对$\frac{\partial \mathbf{p}_s}{\partial \mathbf{\xi}}$的推导不能利用sympy完成，需要手动推导:

$$\begin{align} \frac{\partial \mathbf{p}_s}{\partial \delta \xi} &= \frac{\partial(\mathbf{T}_{sw} \mathbf{p}_w)}{\partial \delta \xi}\\ & = \lim _{\delta \xi \rightarrow 0} \frac{\exp \left(\delta \xi^{\wedge}\right) \exp \left(\xi^{\wedge}\right) \mathbf{p}_w-\exp \left(\xi^{\wedge}\right) \mathbf{p}_w}{\delta \xi}\\ & = \left[\begin{array}{cc}\mathbf{I}_{3 \times 3} & -(\mathbf{R}_{sw} \mathbf{p}_w+\mathbf{t}_{sw})_{3 \times 3}^{\wedge} \\\mathbf{0}_{1 \times 3}^{T} & \mathbf{0}_{1 \times 3}^{T}\end{array}\right] \\ & = \left[\begin{array}{cc}\mathbf{I}_{3 \times 3} & -\left \lfloor \mathbf{p}_s \right \rfloor_{3 \times 3}^{\wedge} \\\mathbf{0}_{1 \times 3}^{T} & \mathbf{0}_{1 \times 3}^{T}\end{array}\right] \end{align}$$
注意：在这里，$\mathbf{p}_s和\mathbf{p}_w$为齐次坐标，因为我们用到了齐次变换矩阵。
若$\mathbf{p}_s$为非齐次坐标形式，则
$$\frac{\partial \mathbf{p}_{s} }{\partial \delta \xi} =\left[\begin{array}{cc}\mathbf{I}_{3 \times 3} & -\left\lfloor\mathbf{p}_{s}\right\rfloor_{3 \times 3}\end{array}\right]$$
故
$$\begin{align}\frac{\partial \mathbf{r}}{\partial \mathbf{\xi}} & = \frac{\partial \mathbf{p}^d_p}{\partial \mathbf{p}_{n}} \frac{\partial \mathbf{p}_n}{\partial \mathbf{p}_{s}} \frac{\partial \mathbf{p}_s}{\partial \mathbf{\xi}}\\&=\left[\begin{matrix}\alpha_{x} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 2 p_{1} y_{n} + 4 p_{2} x_{n} + 1\right) & 2 \alpha_{x} p_{1} x_{n}\\2 \alpha_{y} p_{2} y_{n} & \alpha_{y} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 4 p_{1} y_{n} + 2 p_{2} x_{n} + 1\right)\end{matrix}\right]\\&\ \ \  \cdot  \left[\begin{matrix}\frac{1}{z_{s}} & 0 & - \frac{x_{s}}{z_{s}^{2}}\\0 & \frac{1}{z_{s}} & - \frac{y_{s}}{z_{s}^{2}}\end{matrix}\right] \\&\ \ \  \cdot \left[\begin{array}{cc}\mathbf{I}_{3 \times 3} & -\left\lfloor\mathbf{p}_{s}\right\rfloor_{3 \times 3}\end{array}\right]\end{align}$$

下面推导$\frac{\partial \mathbf{r}}{\partial \mathbf{p}_w}$:
由于$\frac{\partial \mathbf{p}_s}{\partial \mathbf{P}_w}=\mathbf{R}_{sw} = Exp(\phi)$，
则$$\begin{align}\frac{\partial \mathbf{r}}{\partial \mathbf{\xi}} 
& = \frac{\partial \mathbf{p}^d_p}{\partial \mathbf{p}_{n}} \frac{\partial \mathbf{p}_n}{\partial \mathbf{p}_{s}} \frac{\partial \mathbf{p}_s}{\partial \mathbf{p}_w}\\
&=\left[\begin{matrix}\alpha_{x} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 2 p_{1} y_{n} + 4 p_{2} x_{n} + 1\right) & 2 \alpha_{x} p_{1} x_{n}\\2 \alpha_{y} p_{2} y_{n} & \alpha_{y} \left(k_{1} r_{n}^{2} + k_{2} r_{n}^{4} + 4 p_{1} y_{n} + 2 p_{2} x_{n} + 1\right)\end{matrix}\right]\\&\ \ \  \cdot  \left[\begin{matrix}\frac{1}{z_{s}} & 0 & - \frac{x_{s}}{z_{s}^{2}}\\0 & \frac{1}{z_{s}} & - \frac{y_{s}}{z_{s}^{2}}\end{matrix}\right] \\&\ \ \  
\cdot 
\mathbf{R}_{sw}
\end{align}$$